Como podemos usar o poder de Deep Learning para realizar tarefas de NLP? Bom, podemos pensar em aplicações mais complexas como _Speech Recognition_; _Chatbots_ que entendem perguntas complexas, entre outras situações. Contudo, podemos tirar proveito de NLP com Deep Learning de uma forma "relativamente" (áspas intencionais) simples: que é através do _Word2Vec_.

O [_Word2Vec_](https://code.google.com/archive/p/word2vec/) é uma aplicação desenvolvida pela Google por volta de 2013 que nada mais é do que um modelo de _Embedding de palavras_. Para aqueles que queiram uma definição mais formal, essa [apresentação](https://docs.google.com/file/d/0B7XkCwpI5KDYRWRnd1RzWXQ2TWc/edit) pode ser util.

## Embeddings de Palavras

Quando lidamos com palavras em texto, podemos ter uma situação em que temos milhares de classes para prever, uma para cada palavra. Uma abordagem que podemos usar é a conhecida como **one hot encoding**, em que o número de neurônios da nossa Rede Neural equivale ao número de palavras do nosso _dataset_ e ao entrar diferentes palavras, sua posição no vocabulário é setada como 1 e nos demais, 0. Contudo, esse tipo de situação é extreamente ineficiente, já que por exemplo, podemos ter uma situação com apenas um elemento setado como 1 e 50,000 como 0. Computacionalmente é um processo caro.

![One Hot Encoding](img/one-hot.png)

Para resolver esse problema nós usamos o que pode ser chamado de _Word Embedding_._Embeddings_ nada mais são do que uma camada oculta (a camada de _Embedding_) de uma Rede Neural Artificial como essa que a gente acabou de ver. Nos _Embeddings_, não é preciso computar todos os neurônios, mas apenas pegar a linha dos vetores de peso (_embedding weights_). A gente pode fazer isso porquê a multiplicação de um vetor _one hot encoding_ por essa camada irá resultar apenas no valor da linha correspondente ao 1.

![One Hot Multiply](img/one-multiply.png)


Logo, quando tratamos com _Embeddings_, computacionalmente falando não ocorre multiplicação matricial, mas apenas um _lookup_. Lembra que os computadores entendem apenas números? Então, suponha que a palavra _house_ tenha sido encodada na linha 958 enquanto _life_ na 1056. Para obter os valores de peso de _house_ basta olhar na linha 958 da tabela de pesos. Esse processo é conhecido como **_embedding lookup_** e o número de neurônios dessa camada oculta é conhecida como **_embedding dimension_**.

Embeddings não são usados para palavras, apenas. Você pode usá-los para qualquer modelo que tenha um número massivo de classe. Um desses modelos é o _Word2Vec_, que usa modelos de palavras para encontrar representações semânticas das palavras.

## A intuição do Word2Vec

O _Word2Vec_ foi uma aplicação capaz de mostrar que redes neurais conseguem encontrar assimilações entre palavras. Por exemplo:
- **homem** está para **mulher** assim como **rei** está para?

### Para pensar um pouco:

Em seu paper original, Mikolov usou o dataset do _Google News_ que era composto de 3 milhões de palavras e frases, resultando em mais de 100 bilhões de palavras. Nele, foi possível encontrar associações bem interessantes, como:

- Microsoft - Windows ; Google- ?; IBM- ?; Apple -?

![Word2Vec](img/word2vec.png)

### Por debaixo dos panos

![SemanticSpace](img/semanticspace.png)


Sendo assim, é possível encontrar relações bem interessantes, não só limitadas a relações pessoais, como também de verbos e lugares:


![Relationships](img/relationships.png)


### A arquitetura por trás disso

A ideia aqui não é ensinar a como implementar o _Word2Vec_ do zero, mas sim entender toda a intuição por trás dele. Lembra que mais cedo eu falei que estruturas de _Embedding_ nada mais são do que camadas ocultas de Redes Neurais? Então, as duas arquiteturas que implementam o _Word2Vec_ são **Redes Neurais**: o continuous Bag of Words (**CBOW**) e o **Skip-gram**.

![Architetures](img/arc.png)

Como podemos ver, as diferenças entre cada uma das arquiteturas é a seguinte:

-**CBOW**: Dado um contexto (uma janela de palavras), eu tento prever a palavra do meio da janela.

-**Skip-gram**: Dada uma palavra, eu tento prever as palavras em volta (ou o contexto) em que ela está inserida.

Pelo paper, a arquitetura _skipgram_ tende a performar melhor que o _CBOW_.

# TODO


- Dizer o que é o Most Similar
- Falar de Levenisthein

## Preparando os dados

Para a aula de hoje, usaremos apenas o pacote de processamento de texto `gensim`. A ideia é mostrar que ele é uma alternativa ao `scikit-learn` para algumas etapas de pré processamento, mas também é válido dizer que ambos são complementares. Em outras palavras, é possível usar a saída de algum processo do `gensim` no `scikit` e vice versa.

Uma coisa legal do Word2Vec é que ele pode ser considerado como um tipo de aprendizado **não** surpervisionado. Em outras palavras, ele é capaz de aprender de textos que não possuam um rótulo (ou label). Para ilustrar isso, para o treinamento aqui, além de usarmos o _labeledTrainData.tsv_, também usaremos o _unlabeledTrainData.tsv_, que contém 50 mil reviews adicionais, sem nenhuma label.

In [19]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

In [20]:
train.shape

(25000, 3)

In [21]:
test.shape

(25000, 2)

In [22]:
unlabeled_train.shape

(50000, 2)

## Pré Processando o texto

É interessante realizarmos o pré processamento dos textos que vamos trabalhar, igual fizemos na aula passada. Contudo, diferente do modelo **BoW** que vimos, o _Word2Vec_ **leva** em consideração o contexto das palavras. Sendo assim, talvez não seja muito legal removermos as *stop words*, assim como reduzir as palavras ao seu radical (stemming ou lemma) pode não ser desejável.

Contudo, um pré processamento interessante é remover as URL's, uma vez que a chance de um site ser citado em um review é baixo e não estamos muito interessados na relação entre sites, mas no contexto em que levou a pessoa a citar alguma fonte.

In [60]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

#exemplo de uma regex não trivial
#RE_URL = re.compile(r'(http[s]?://)?(www)?.([a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\.(com|net|org)(.[A-Za-z]{2})?')
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    
    # 1. Ignore URLs
    review_text = re.sub("(?:https?):\/\/[\n\S]+","<URL>", review)
    # 2. Remove HTML
    review_text = BeautifulSoup(review_text, "html5lib").get_text()
    #  
    # 3. Remove non-letters
    review_text = re.sub("\W+"," ", review_text)


    #
    # 4. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 5. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 6. Return a list of words
    return(words)

Uma vez limpa, agora precisamos converter os nossos dados para o tipo de dado que a biblioteca do `gensim` aceite. O _Word2Vec_ recebe como entrada sentenças únicas, cada uma formada por uma lista de palavras. Ou seja, precisamos de uma lista de listas.

Definir onde começa ou onde acaba uma sentença não é algo muito claro. Existem sentenças que podem terminar com "?";".";"!" entre outras. Por conta disso, usaremos o pacote punkt tokenizer do NLTK para fazer esse tipo de split pra gente. 


In [61]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from tqdm import tqdm

In [62]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review,remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = sent_tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [63]:
sentences = []  # Initialize an empty list of sentences

for review in tqdm(train["review"], total=len(train["review"])):
    #fezer com extend e comparar
    sentences += review_to_sentences(review)

print ("Starting Unlabeled Sentences")

for review in tqdm(unlabeled_train["review"], total=len(unlabeled_train["review"])):
    sentences += review_to_sentences(review)


  1%|          | 219/25000 [00:03<06:41, 61.65it/s]/Users/abarbosa/miniconda3/envs/somostera/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)

  0%|          | 9/50000 [00:00<09:21, 89.08it/s]

Starting Unlabeled Sentences


 62%|██████▏   | 31190/50000 [09:15<15:36, 20.09it/s]/Users/abarbosa/miniconda3/envs/somostera/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
100%|██████████| 50000/50000 [14:57<00:00, 55.74it/s]


In [64]:
len(sentences)

795538

In [65]:
sentences[0]

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again']

In [66]:
sentences[1]

['maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent']

## Treinando o modelo Word2Vec

Com a lista de sentenças devidamente parseada, basta treinarmos o nosso modelo. Aqui tem uma lista dos parâmetros especificando o que cada uma coisa faz.

- **Architecture**: As opções de arquitetura são os algoritmos possíveis para a realização do Word2Vec. As opções aqui são o _skip-gram_ (default) e o _continuous bag of words_ (_cbow_).

- **Training algorithm**: _Hierarchical softmax_ (default) ou _negative sampling_. Aqui, o default funcionou bem.

- **Downsampling of frequent words**: A documentação do Google sugere valores entre .00001 and .001. Aqui, algo epor volta de 0.001 parece que melhorou o modelo final.

- **Word vector dimensionality**: Aqui a ideia de _feature_ é parecida com o modelo que vimos do _BoW_, mas cada coluna não necessariamente é uma palavra. Logo, mais _features_ podem resultar em tempos maiores de processamento, mas nem sempre em modelos ideiais. O default aqui é 300.

- **Context / window size**: Quantas janelas de contexto o algoritmo de treinamento deve levar em consideração. 10 parece funcionar bem para o softmax hierarquico (quanto mais melhor, até certo ponto).

- **Worker threads**: Numero de processos para correrem em paraleolo. Isso varia de máquina para máquina, mas algo entre 4 e 6 deve funcionar na maioria dos sistemas.

- **Minimum word count**: Isso ajuda a limitar o tamanho do vocabulario de palavras significativas. Qualquer palavra que não ocorre ao menos esse número de vezes considerando **todos** os documentos é ignorada. Valores razoáveis são entre 10 e 100. Nesse caso, dado que cada filme ocorre 30 vezes, o valor setado de _minimum word count_ é de 40. Quanto maior o vocabulário, maior o tempo de execução é impactado.

In [67]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2017-11-28 23:27:37,159 : INFO : collecting all words and their counts
2017-11-28 23:27:37,161 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-11-28 23:27:37,294 : INFO : PROGRESS: at sentence #10000, processed 227183 words, keeping 18050 word types


Training model...


2017-11-28 23:27:37,386 : INFO : PROGRESS: at sentence #20000, processed 454403 words, keeping 25345 word types
2017-11-28 23:27:37,478 : INFO : PROGRESS: at sentence #30000, processed 675010 words, keeping 30508 word types
2017-11-28 23:27:37,562 : INFO : PROGRESS: at sentence #40000, processed 902687 words, keeping 34903 word types
2017-11-28 23:27:37,631 : INFO : PROGRESS: at sentence #50000, processed 1123085 words, keeping 38385 word types
2017-11-28 23:27:37,699 : INFO : PROGRESS: at sentence #60000, processed 1345767 words, keeping 41405 word types
2017-11-28 23:27:37,765 : INFO : PROGRESS: at sentence #70000, processed 1570193 words, keeping 44067 word types
2017-11-28 23:27:37,836 : INFO : PROGRESS: at sentence #80000, processed 1790610 words, keeping 46494 word types
2017-11-28 23:27:37,909 : INFO : PROGRESS: at sentence #90000, processed 2016019 words, keeping 48972 word types
2017-11-28 23:27:37,979 : INFO : PROGRESS: at sentence #100000, processed 2239092 words, keeping 51

2017-11-28 23:27:43,866 : INFO : PROGRESS: at sentence #740000, processed 16643523 words, keeping 123026 word types
2017-11-28 23:27:43,962 : INFO : PROGRESS: at sentence #750000, processed 16863120 words, keeping 123677 word types
2017-11-28 23:27:44,047 : INFO : PROGRESS: at sentence #760000, processed 17083737 words, keeping 124327 word types
2017-11-28 23:27:44,131 : INFO : PROGRESS: at sentence #770000, processed 17312165 words, keeping 125128 word types
2017-11-28 23:27:44,225 : INFO : PROGRESS: at sentence #780000, processed 17543573 words, keeping 125864 word types
2017-11-28 23:27:44,302 : INFO : PROGRESS: at sentence #790000, processed 17771837 words, keeping 126562 word types
2017-11-28 23:27:44,342 : INFO : collected 127017 word types from a corpus of 17895591 raw words and 795538 sentences
2017-11-28 23:27:44,343 : INFO : Loading a fresh vocabulary
2017-11-28 23:27:44,458 : INFO : min_count=40 retains 16717 unique words (13% of original 127017, drops 110300)
2017-11-28 23:

2017-11-28 23:28:50,197 : INFO : PROGRESS: at 96.37% examples, 948898 words/s, in_qsize 7, out_qsize 0
2017-11-28 23:28:51,199 : INFO : PROGRESS: at 97.90% examples, 949367 words/s, in_qsize 7, out_qsize 0
2017-11-28 23:28:52,207 : INFO : PROGRESS: at 99.43% examples, 949761 words/s, in_qsize 7, out_qsize 0
2017-11-28 23:28:52,563 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-11-28 23:28:52,570 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-11-28 23:28:52,576 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-11-28 23:28:52,583 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-11-28 23:28:52,584 : INFO : training on 89477955 raw words (64279313 effective words) took 67.7s, 949978 effective words/s
2017-11-28 23:28:52,585 : INFO : precomputing L2-norms of word weight vectors
2017-11-28 23:28:52,763 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2017-11-2

## Explorando os resultados

A função `doesnt_match` tentará deduzir qual palavra em um set é a mais _distinta_ das outras.

De forma análoga, a função `most_similar` tentará deduzir qual palavra em um set é a mais _semelhante_ das outras.


## Mas como definir similaridade? Que uma palavra é similar à outra?

A métrica mais famosa para isso é a _Similaridade de Coscenos_ ou _Cosine Similarity_.

Vamos voltar à definição de espaço semantico dada no começo da aula.

![Male-Female](img/male-female.png)

Olhando um pouco debaixo dos panos (porque na situação real é como se estivessemos olhando para um vetor de 300 dimensões, o que é muito abstrato), vamos projetar **apenas** essa situação de _Male-Female_


![Male-Female](img/sematic-male-female.png)


Ao olharmos esse espaço, vemos claramente que as palavras semânticamente próximas a _male_ também são próximas entre si. O mesmo acontece com _female_. Mas como que a gente encontra esse *valor*? Basta calcularmos o **cosceno** entre os vetores que formam essas palavras !

Lembrando um pouquinho de matemática da faculdade, dado o produto escalar entre dois vetores e suas normas eu consigo encontrar o cosceno formado entre os dois:

$A \cdot B = \lVert A \rVert \lVert B \rVert \cos\theta   $

![Cosine](img/cosine-similarity.png)


Em NLP essa métrica é muito comum quando queremos calcular a similaridade entre documentos.

## Entendendo a representação numérica das palavras

Agora nós treinamos um modelo que consegue entender a relação semântica entre as palavras. Para algumas aplicações é exatamente isso que a gente quer, mas o que podemos fazer com isso? Os vetores de palavras criados pelo nosso modelo foram armazenados em um array `numpy` chamado `syn0`

In [78]:
from gensim.models import Word2Vec 
model = Word2Vec.load("300features_40minwords_10context")
type(model.wv.syn0)

2017-11-28 23:36:08,530 : INFO : loading Word2Vec object from 300features_40minwords_10context
2017-11-28 23:36:08,887 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2017-11-28 23:36:08,888 : INFO : setting ignored attribute syn0norm to None
2017-11-28 23:36:08,889 : INFO : setting ignored attribute cum_table to None
2017-11-28 23:36:08,890 : INFO : loaded 300features_40minwords_10context


numpy.ndarray

In [79]:
model.wv.syn0.shape

(16717, 300)

Lembre-se que a gente limitou o número de ocorrências de palavras para um mínimo de 40. Isso justifica a menor quantidade de vocabulário, sendo formado por 16717 palavras.

In [75]:
model["flower"]

array([ -3.10837384e-02,   4.03764322e-02,   4.10359241e-02,
         5.15633961e-03,  -5.40248640e-02,  -7.78134391e-02,
         1.35028914e-01,   7.21310675e-02,   3.28026302e-02,
         2.29188558e-02,   2.05275626e-03,   3.21877468e-03,
         3.96432392e-02,  -1.32296890e-01,   1.10449947e-01,
        -3.37153859e-02,  -7.52574503e-02,  -8.52601156e-02,
        -8.54553189e-03,   4.08086330e-02,   4.99160737e-02,
        -5.79162985e-02,  -4.89986548e-03,  -5.31144291e-02,
         1.21043175e-01,  -4.36744876e-02,  -2.73520648e-02,
        -5.54944053e-02,  -3.95381078e-02,  -4.12611589e-02,
         1.68824494e-02,  -3.27361412e-02,   8.19914192e-02,
         2.42416244e-02,   3.13074738e-02,   6.38195872e-02,
        -4.22389247e-02,   1.01815984e-02,  -7.76159763e-02,
         8.76649246e-02,   1.06198631e-01,  -7.59951025e-02,
        -1.18816376e-03,  -8.79209787e-02,  -4.08728644e-02,
         5.11247665e-02,   8.62793345e-03,   4.73641492e-02,
         3.71092930e-02,

In [76]:
len(model["flower"])

300

## De palavras para reviews 

Um problema que temos ao lidar com o dataset do IMDB é que os reviews tem tamanho variado. Precisaríamos, então, encontrar uma forma de pegar esses vetores de palavras e transformá-los em um conjunto de features que tem o mesmo tamanho para o mesmo _review_.

Já que todas as palavras são representadas por um vetor de 300 dimensões, nós podemos usar operações vetoriais para combinar palavras. Por exemplo, podemos pegar todas as palavras que compõe um review e somá-las ou tirar a média. No caso, vamos tirar a média (contudo, aqui nós não precisamos mais do contexto das palavras e, então, podemos remover stopwords).

In [85]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in tqdm(reviews,total=len(reviews)):

       # 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [86]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in tqdm(train["review"], total=len(train["review"])):
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in tqdm(test["review"], total=len(test["review"])):
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

  0%|          | 45/25000 [00:00<00:56, 443.83it/s]

Creating average feature vecs for test reviews


100%|██████████| 25000/25000 [00:30<00:00, 807.71it/s]


Aqui, como a ideia é usar o vetor de features como treino, estamos lidando novamente com um aprendizado **surpervisionado** e, então, usaremos apenas os dados com _label_.

In [89]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"],"review":test["review"],"sentiment":result} )

output

,id,review,sentiment
0,"""12311_10""","""Naturally in a film who's main themes are of ...",1
1,"""8348_2""","""This movie is a disaster within a disaster fi...",0
2,"""5828_4""","""All in all, this is a movie for kids. We saw ...",1
3,"""7186_2""","""Afraid of the Dark left me with the impressio...",0
4,"""12128_7""","""A very accurate depiction of small time mob l...",1
5,"""2913_8""","""...as valuable as King Tut's tomb! (OK, maybe...",1
6,"""4396_1""","""This has to be one of the biggest misfires ev...",0
7,"""395_2""","""This is one of those movies I watched, and wo...",0
8,"""10616_1""","""The worst movie i've seen in years (and i've ...",0
9,"""9074_9""","""Five medical students (Kevin Bacon, David Lab...",1


## Mas analisando o Word2Vec 

Os resultados foram melhores que o BoW?

Se analisarmos de uma forma minuciosa, vemos que os resultados do BoW foram **um pouco melhores** que o do Word2Vec, mas porquê isso ocorre?

O fato da gente utilizar a média para criar as sentenças faz com que nós percamos a ordem das palavras (além de perdermos informação ao resumir os dados por meio da média), fazendo com que a abordagem seja muito parecida com o conceito do Bag Of Words, de forma que os métodos até sejam considerados equivalentes.

Algumas coisas para melhorar:
- Dados, dados dados. Lembre-se que o paper original usou um corpus de mais de um **bilhão** de palavras enquanto os nossos dados tinham aproximadamente 18 **milhões** de palavras. É possível usar vetorer pré treinados para usar em nossos dados e talvez isso resulte em resultados mais interessantes.
